In [57]:
import os
import operator
import numpy as np
import pandas as pd
import pickle
from scipy import stats
from tqdm import tqdm
from skmob.utils import gislib
from skmob.utils.gislib import getDistanceByHaversine
from skmob.utils import constants
from collections import defaultdict, Counter
import editdistance
from sklearn_extra.cluster import KMedoids
from sklearn.metrics import silhouette_score
from multiprocessing.pool import ThreadPool
import torch
import random
import argparse
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from torch.nn import Transformer
import math
from math import sqrt, sin, cos, pi, asin, pow, ceil
from random import random, uniform, choice
from skmob.measures.evaluation import common_part_of_commuters

## 1. 活动序列生成
- 加载数据

In [31]:
data = data[data["uid"]<=2000]
data.head()

,uid,datetime,tile,lat,lng,day
0,0,2021-11-01 00:00:00,1720,22.729155,114.004822,1
1,0,2021-11-01 01:00:00,1720,22.729155,114.004822,1
2,0,2021-11-01 02:00:00,1720,22.729155,114.004822,1
3,0,2021-11-01 03:00:00,1720,22.729155,114.004822,1
4,0,2021-11-01 04:00:00,1720,22.729155,114.004822,1


- 识别居住地、工作地、其他位置

In [6]:
def _location_individual(traj, start_time='21:00', end_time='06:00', label='home', location_column='tile'):
    # Filter trajectory data based on time
    night_visits = traj.set_index(pd.DatetimeIndex(traj.datetime)).between_time(start_time, end_time)
    if len(night_visits) != 0:
        traj = night_visits
    
    # Identify potential home/work locations
    candidates = traj.groupby([location_column]).count().sort_values(by='datetime', ascending=False)
    
    if (len(candidates) > 0):
        location = candidates.iloc[0].name

        # If label is 'work', check if work duration is greater than home duration * 0.4
        if label == 'work' and len(traj.groupby([location_column]).count()) > 1:
            home = hl_df[hl_df['uid'] == traj['uid'].iloc[0]][location_column].values
            home_count = candidates.iloc[0]
            work_count = candidates.iloc[1]
            if location == home and work_count.unique() > home_count.unique() * 0.4:
                location = work_count.name

        return location

def location_infer(traj, label='home', location_column='cluster'):
    # Define time intervals based on label
    if label == 'home':
        start_time, end_time = '21:00', '06:00'
    elif label == 'work':
        start_time, end_time = '09:00', '18:00'
    
    # If 'uid' column is not present in the TrajDataFrame
    if 'uid' not in traj.columns:
        return pd.DataFrame([_location_individual(traj, start_time=start_time, end_time=end_time, label=label, location_column=location_column)], columns=[location_column])
    
    # Group by user ID and infer locations
    df = traj.groupby('uid').apply(lambda x: _location_individual(x, start_time=start_time, end_time=end_time, label=label, location_column=location_column))

    return pd.DataFrame(df.to_list(), index=df.index).reset_index().rename(columns={0: location_column}) # Convert series to dataframe

data['day'] = pd.to_datetime(data['datetime'],format='%Y-%m-%d %H:%M:%S').dt.dayofweek+1
# Infer home and work locations
hl_df = location_infer(data[data['day']<=5], label='home', location_column='tile')
wl_df = location_infer(data[data['day']<=5], label='work', location_column='tile')

C:\Users\86152\AppData\Local\Temp/ipykernel_67440/789971869.py:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = traj.groupby('uid').apply(lambda x: _location_individual(x, start_time=start_time, end_time=end_time, label=label, location_column=location_column))
C:\Users\86152\AppData\Local\Temp/ipykernel_67440/789971869.py:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = traj.groupby('uid').apply

- 轨迹转为活动序列

In [7]:
def relabel(tdf, uid_column='uid', location_column='tile', datetime_column='datetime'):
    # Convert dataframe to list of lists
    dtime_cluster = tdf[[uid_column, datetime_column, location_column]].values.tolist()

    # Extract UID from the first row
    uid = dtime_cluster[0][tdf.columns.get_loc(uid_column)]

    # Check if both home and work locations are identified for the user
    if len(hl_df[hl_df[uid_column] == uid]) > 0 and len(wl_df[wl_df[uid_column] == uid]) > 0:
        location2order = defaultdict(int)
        home = str(hl_df[hl_df[uid_column] == uid][location_column].values[0])
        work = str(wl_df[wl_df[uid_column] == uid][location_column].values[0])

        lendata = len(dtime_cluster)
        
        cnt = 1
        # Assign order to other locations
        for i in range(lendata):
            location = dtime_cluster[i][tdf.columns.get_loc(location_column)]
            if location2order[location] == 0 and str(location) != home and str(location) != work:
                location2order[location] = cnt
                cnt += 1
        
        stay_locations = []
        # Relabel locations as home, work, or numbered locations
        for i in range(lendata):
            location = dtime_cluster[i][tdf.columns.get_loc(location_column)]
            dtime = dtime_cluster[i][tdf.columns.get_loc(datetime_column)]

            if str(location) == home:
                label = 'H'
            elif str(location) == work:
                label = 'W'
            else:
                label = str(location2order[location])

            stay_locations.append([uid, dtime, label])
            
        return pd.DataFrame(stay_locations, columns=[uid_column, datetime_column, 'activity'])

# Apply relabel function to each user's trajectory data
hw = data.groupby(["uid"], group_keys=False).apply(lambda x: relabel(x, uid_column='uid', location_column='tile', datetime_column='datetime'))
activity_sequence = hw['activity'].values.reshape(-1, 24*7)
activity_sequence

C:\Users\86152\AppData\Local\Temp/ipykernel_67440/2043431700.py:42: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  hw = data.groupby(["uid"], group_keys=False).apply(lambda x: relabel(x, uid_column='uid', location_column='tile', datetime_column='datetime'))


array([['H', 'H', 'H', ..., 'H', 'H', 'H'],
       ['H', 'H', 'H', ..., 'H', 'H', 'H'],
       ['H', 'H', 'H', ..., 'H', 'H', 'H'],
       ...,
       ['H', 'H', 'H', ..., '9', '9', '9'],
       ['H', 'H', 'H', ..., 'H', 'H', 'H'],
       ['H', 'H', 'H', ..., 'H', 'H', 'H']], dtype=object)

- 活动序列聚类

In [8]:
# Function to calculate edit distance between strings
def func(X, i):
    ls = []
    for j in range(int(i + 1)):
        ls.append(editdistance.eval(X[i], X[j]))
    return ls

# Function to calculate the edit distance matrix
def distance(X):
    n = len(X)
    dists = []
    pool = ThreadPool(16)
    for i in tqdm(range(n)):
        dists.append(pool.apply_async(func, args=(X, i)).get())

    dists = sorted(dists, key=len)
    for i, item in enumerate(dists):
        dists[i].extend(0 for _ in range(n - i - 1))
    dists = np.array(dists)
    
    # Make the matrix symmetric
    i_upper = np.triu_indices(n)
    dists[i_upper] = dists.T[i_upper]
    
    return dists

def test_kmedoids(activity_sequence, dists, num_clusters):
    scores, distortions = [], []
    cluster_results = {}  # Dictionary to store cluster results
    
    for num_cluster in tqdm(range(2, num_clusters)):
        # Initialize KMedoids with precomputed distance matrix
        km = KMedoids(n_clusters=num_cluster, metric='precomputed', init='k-medoids++')
        model = km.fit(dists)
        
        # Record inertia (sum of squared distances) as distortion
        distortions.append(model.inertia_)
        
        # Predict cluster labels
        y_pred = km.fit_predict(dists)
        
        # Convert distance matrix to float32
        dists = np.array(dists, dtype="float32")
        
        # Calculate silhouette score if more than one cluster is formed
        if len(set(y_pred)) > 1:
            score = silhouette_score(dists, y_pred, metric="precomputed")
            scores.append(score)
            print("Silhouette Score:", score)

        # Convert distance matrix back to list for the next iteration
        dists = dists.tolist()

        # Save cluster results
        cluster_results[num_cluster] = y_pred

    # Determine the best number of clusters based on the highest silhouette score
    best_cluster = 2 + scores.index(max(scores))
    print("Best number of clusters:", best_cluster)

    # Perform KMedoids clustering with the best number of clusters
    km = KMedoids(n_clusters=best_cluster, metric='precomputed', init='k-medoids++')
    y_pred = km.fit_predict(dists)
    
    # Convert non-'H' and non-'W' elements in activity sequence to ASCII codes
    for i in range(len(activity_sequence)):
        for j in range(len(activity_sequence[i])):
            if activity_sequence[i][j] != 'H' and activity_sequence[i][j] != 'W':
                activity_sequence[i][j] = str(ord(activity_sequence[i][j]))

    # Save clustering results for each cluster

     # Visualize the clustering result
    for cluster_index in tqdm(range(best_cluster)):
        cluster_list = []
        for activity in np.array(activity_sequence)[y_pred == cluster_index]:
            cluster_list.append(activity.tolist())
        cluster_results[cluster_index] = cluster_list
    return cluster_results, best_cluster


num_clusters = 15
for i in range(len(activity_sequence)):
    for j in range(len(activity_sequence[i])):
        if activity_sequence[i][j] != 'H' and activity_sequence[i][j] != 'W':
            activity_sequence[i][j] = chr(int(activity_sequence[i][j]))
dist = distance(activity_sequence) 
dict_sequence, best_cluster = test_kmedoids(activity_sequence, dist, num_clusters)

  8%|██████▍                                                                            | 1/13 [00:00<00:02,  4.17it/s]

Silhouette Score: 0.511068


 15%|████████████▊                                                                      | 2/13 [00:00<00:04,  2.32it/s]

Silhouette Score: 0.4782793


 23%|███████████████████▏                                                               | 3/13 [00:01<00:04,  2.04it/s]

...

 92%|███████████████████████████████████████████████████████████████████████████▋      | 12/13 [00:06<00:00,  1.88it/s]

Silhouette Score: 0.40179172


100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:06<00:00,  1.94it/s]

Silhouette Score: 0.3075749
Best number of clusters: 2



100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 147.50it/s]


- 各聚类生成序列

In [ ]:
from tqdm import tqdm
import numpy as np
import pickle
import torch
import random
import argparse
import math
import torch.nn as nn
import pandas as pd
from collections import Counter
from torch.utils.data import Dataset, DataLoader

# Define a class for time-based dataset
class Timeset():
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        """Returns the number of samples in the dataset"""
        return len(self.dataset)
    
    def __getitem__(self, index):
        """Returns the index-th sample from the dataset; input variables first, output variables last"""
        return (torch.tensor(self.dataset[index]), 
                torch.tensor(self.dataset[index][:-1]),
                torch.tensor(self.dataset[index][1:]))

# Define a class for the main dataset
class Dataset():
    def __init__(self, dataset):
        self.words = np.ravel(np.array(dataset))
        self.uniq_words = self.get_uniq_words()
        
        # Create mappings from word to index and vice versa
        self.index_to_word = {index: word for index, word in enumerate(self.uniq_words)}
        self.word_to_index = {word: index for index, word in enumerate(self.uniq_words)}

        # Convert dataset to index sequences
        self.words_indexes = [[self.word_to_index[w] for w in activity] for activity in dataset]

    def get_uniq_words(self):
        word_counts = Counter(self.words)
        return sorted(word_counts, key=word_counts.get, reverse=True)
    
    def __len__(self):
        """Returns the number of samples in the dataset"""
        return len(self.words_indexes)
    
    def __getitem__(self, index):
        """Returns the index-th sample from the dataset; input variables first, output variables last"""
        return (torch.tensor(self.words_indexes[index]),
                torch.tensor(self.words_indexes[index][:-1]),
                torch.tensor(self.words_indexes[index][1:]))

# Define positional encoding for the Transformer model
class PositionalEncoding(nn.Module):
    "Implement the positional encoding function."

    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Initialize the positional encoding matrix of shape (max_len, d_model)
        pe = torch.zeros(max_len, d_model)
        # Initialize a tensor [[0, 1, 2, 3, ...]]
        position = torch.arange(0, max_len).unsqueeze(1)
        # Calculate the values for sine and cosine terms used in the positional encoding
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
        )
        # Compute PE(pos, 2i)
        pe[:, 0::2] = torch.sin(position * div_term)
        # Compute PE(pos, 2i+1)
        pe[:, 1::2] = torch.cos(position * div_term)
        # Add a batch dimension for easier calculation
        pe = pe.unsqueeze(0)
        # Register the positional encoding tensor as a buffer (non-parameter tensor)
        self.register_buffer("pe", pe)

    def forward(self, x):
        """
        x: the input tensor after embedding, e.g., (1, 7, 128) where batch size is 1, 7 words, and word dimension is 128
        """
        # Add the positional encoding to the input tensor.
        x = x + self.pe[:, : x.size(1)].requires_grad_(False)
        return self.dropout(x)

# Define the Transformer model
class Model(Transformer):
    def __init__(self, dataset):
        super(Model, self).__init__(
            d_model=512, # Input and output dimensions
            nhead=8, # Number of attention heads
            num_encoder_layers=3, # Number of encoder layers
            num_decoder_layers=3, # Number of decoder layers
            dim_feedforward=1024, # Dimension of the feedforward network
            dropout=0.2, # Dropout rate
            batch_first=True
        )
        self.loc_dim = 256
        self.tim_dim = 256
        
        # Initialize embedding matrices
        self.emb_loc = nn.Embedding(num_embeddings=n_vocab, embedding_dim=self.loc_dim)
        self.emb_tim = nn.Embedding(num_embeddings=168, embedding_dim=self.tim_dim)
        
        # Define positional encoding
        self.positional_encoding = PositionalEncoding(d_model=512, dropout=0.2)
        
        self.fc = nn.Linear(self.d_model, n_vocab) # Fully connected layer
    
    def forward(self, src_loc, src_tim, tgt_loc, tgt_tim, src_mask=None, tgt_mask=None):
        
        # Concatenate embedding matrices
        src_loc = torch.squeeze(self.emb_loc(src_loc))
        src_tim = self.emb_tim(src_tim)
        src = torch.cat((src_loc, src_tim), 2) # Source sequence embedding

        tgt_loc = torch.squeeze(self.emb_loc(tgt_loc))
        tgt_tim = self.emb_tim(tgt_tim)
        tgt = torch.cat((tgt_loc, tgt_tim), 2) # Source sequence embedding

        # Add positional information to src and tgt tokens
        src = self.positional_encoding(src)
        tgt = self.positional_encoding(tgt)
        tgt_mask = nn.Transformer.generate_square_subsequent_mask(tgt_loc.size()[1]).to(device)
        output = super(Model, self).forward(src, tgt, src_mask=src_mask, tgt_mask=tgt_mask) # Transformer encoding-decoding
        
        output = self.fc(output) # Fully connected layer
        
        return output

# Define the training function
def train(dataset, tim, model, args):

    model.train()

    # Split the dataset into training and validation sets
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

    # Create data loaders
    train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, drop_last=True)
    val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, drop_last=True)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    
    timeloader = DataLoader(tim, batch_size=args.batch_size)
    src_tim = next(iter(timeloader))[0]
    src_tim = src_tim.to(device)

    tgt_tim = next(iter(timeloader))[1]
    tgt_tim = tgt_tim.to(device)
    
    # Early stopping parameters
    patience = 20 # Number of allowed epochs without improvement on validation set
    min_delta = 1e-6 # Minimum threshold for improvement
    best_loss = float('inf') # Record the best validation loss
    best_model = None # Record the best model parameters
    
    for epoch in range(args.max_epochs):
        
        # Training mode
        model.train()
        train_loss = 0 # Record training set loss
        
        for batch, (x, y, z) in enumerate(train_dataloader):
            x = x.to(device)
            y = y.to(device)
            z = z.to(device)

            optimizer.zero_grad()

            z_pred = model(x, src_tim, y, tgt_tim)
            loss = criterion(z_pred.transpose(1, 2), z)

            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            
        train_loss /= len(train_dataloader) # Compute average training set loss
        
        # Validation mode
        model.eval()
        val_loss = 0 # Record validation set loss
        
        with torch.no_grad():
            for batch, (x, y, z) in enumerate(val_dataloader):
                x = x.to(device)
                y = y.to(device)
                z = z.to(device)

                z_pred = model(x, src_tim, y, tgt_tim)
                loss = criterion(z_pred.transpose(1, 2), z)

                val_loss += loss.item()
                
        val_loss /= len(val_dataloader) # Compute average validation set loss
        
        print({ 'epoch': epoch, 'train_loss': train_loss, 'val_loss': val_loss })
        
        # Check for early stopping
        if val_loss < best_loss - min_delta: # If there's a significant improvement in validation loss
            best_loss = val_loss # Update the best loss
            best_model = model.state_dict() # Save the best model parameters
            patience = 20 # Reset patience
        else: # If there's no significant improvement in validation loss
            patience -= 1 # Decrease patience
            if patience == 0: # If patience runs out
                print('Early stopping') 
                break # Stop training
                
    # Load the best model parameters
    model.load_state_dict(best_model)
    
    # Evaluate the final model performance on the test set
    # Not implemented...

# Define the prediction function
def predict(dataset, tim, model, args, next_words=169):
    model.eval()

    dataloader = DataLoader(dataset, batch_size=args.batch_size, drop_last=True)
    
    new_words = []
    
    timeloader = DataLoader(tim, batch_size=args.batch_size)
    src_tim = next(iter(timeloader))[0]
    src_tim = src_tim.to(device)

    tgt_tim = next(iter(timeloader))[1]
    tgt_tim = tgt_tim.to(device)
    
    with torch.no_grad():
        for batch, (x, y, z) in tqdm(enumerate(dataloader)):
            x = x.to(device)
            src_word = x.cpu()
            src_time = src_tim.cpu()
            
            y = y.to(device)
            tgt_word = y.cpu()
            tgt_time = tgt_tim.cpu()
            
            for i in range(next_words):
                y_pred = model(torch.tensor(src_word.to(device)), torch.tensor(src_time.to(device)), torch.tensor(tgt_word[:, i:].to(device)), torch.tensor(tgt_time[:, i:].to(device)))
                
                col = np.array([], dtype=int)
                tim = np.array([], dtype=int)
                
                for row in range(len(y_pred)):
                    last_word_logits = y_pred[row][-1]

                    p = torch.nn.functional.softmax(last_word_logits, dim=0).cpu().detach().numpy()
                    word_index = np.random.choice(len(last_word_logits), p=p)
                    col = np.append(col, word_index)
                    tim = np.append(tim, i)
                
                if i == 0:
                    predict = col[:, np.newaxis]
                else:
                    predict = np.concatenate((predict, col[:, np.newaxis]), axis=1)

                tim = torch.tensor(tim[:, np.newaxis])
                tgt_time = torch.cat((tgt_time, tim), dim=1)
                    
                col = torch.tensor(col[:, np.newaxis])
                tgt_word = torch.cat((tgt_word, col), dim=1)

            new_words.extend(predict.tolist())

    return np.array(new_words)[:, 1:].tolist()        

# Iterate over clusters and train models
generated_sequence = []
for cluster_index in range(best_cluster):
    activity_set = dict_sequence[cluster_index]
    
    random.shuffle(activity_set)
    ravel = np.ravel(np.array(activity_set))
    vocab = set(np.unique(ravel))
    n_vocab = len(vocab)

    parser = argparse.ArgumentParser()
    parser.add_argument('--max-epochs', type=int, default=5000)
    parser.add_argument('--batch-size', type=int, default=64)
    parser.add_argument('--sequence-length', type=int, default=167)
    args = parser.parse_args(args=[])
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 

    tim = [[i for i in range(168)]for item in range(args.batch_size)]
    tim  = Timeset(tim)
    
    dataset = Dataset(activity_set)

    model = Model(activity_set).to(device) 

    train(dataset, tim, model, args)
    words = predict(dataset, tim, model, args)

    dicts = dataset.index_to_word
    generated_sequence.extend([[dicts[item] for item in seq] for seq in tqdm(words)])

## 3. UO模型参数

In [29]:
#!/usr/bin/python
#coding:utf-8
import geopandas as gpd
import numpy as np 
import pandas as pd

# 人口分布数据选择：
def load_spatial_tessellation(tessellation):
    # relevance: population
    M = 0
    spatial_tessellation = {}
    f = np.array(tessellation)

    for line in f:
        i = int(line[0])
        relevance = int(line[3])
        if relevance == 0:
            relevance = 1e-1
        spatial_tessellation[i] = {'lat': float(line[1]),
                                    'lon': float(line[2]),
                                    'relevance': relevance}

        M += relevance

    return spatial_tessellation, M

# load the spatial tessellation
tessellation = gpd.read_file('data/sz_1km.shp')
spatial_tessellation, M = load_spatial_tessellation(tessellation)
spatial_tessellation

{0: {'lat': 22.3988432348, 'lon': 113.803537169, 'relevance': 0.1},
 1: {'lat': 22.3986745674, 'lon': 113.813240804, 'relevance': 0.1},
 2: {'lat': 22.3985053186, 'lon': 113.822944347, 'relevance': 0.1},
 3: {'lat': 22.4082022304, 'lon': 113.784309491, 'relevance': 0.1},
 4: {'lat': 22.4080346501, 'lon': 113.794013937, 'relevance': 0.1},
 5: {'lat': 22.4078664886, 'lon': 113.803718289, 'relevance': 0.1},
 6: {'lat': 22.4076977459, 'lon': 113.813422549, 'relevance': 0.1},
 7: {'lat': 22.407528422, 'lon': 113.823126717, 'relevance': 0.1},
 8: {'lat': 22.4172256222, 'lon': 113.784489454, 'relevance': 0.1},
 9: {'lat': 22.4170579674, 'lon': 113.794194524, 'relevance': 0.1},
 10: {'lat': 22.416889731, 'lon': 113.803899502, 'relevance': 0.1},
 11: {'lat': 22.416720913, 'lon': 113.813604388, 'relevance': 0.1},
 12: {'lat': 22.4260812731, 'lon': 113.794375204, 'relevance': 0.1},
 13: {'lat': 22.4259129619, 'lon': 113.804080807, 'relevance': 0.1},
 14: {'lat': 22.4425907353, 'lon': 113.88209517

In [111]:
import operator
import os.path
import pickle
from tqdm import tqdm
import numpy as np
import pandas as pd

def earth_distance(lat_lng1, lat_lng2):
    """
    Calculate the distance between two points on Earth's surface using Haversine formula.

    Args:
        lat_lng1 (tuple): Latitude and longitude of the first point.
        lat_lng2 (tuple): Latitude and longitude of the second point.

    Returns:
        float: The distance between the two points in kilometers.
    """
    lat1, lng1 = [l * np.pi / 180 for l in lat_lng1]
    lat2, lng2 = [l * np.pi / 180 for l in lat_lng2]
    dlat, dlng = lat1 - lat2, lng1 - lng2
    ds = 2 * np.arcsin(np.sqrt(np.sin(dlat / 2.0) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlng / 2.0) ** 2))
    return 6371.01 * ds  # Spherical Earth radius

def radiation_od_matrix(spatial_tessellation, alpha=0, beta=1):
    """
    Compute the origin-destination matrix using the radiation model.

    Args:
        spatial_tessellation (dict): Dictionary containing spatial tessellation data.
        alpha (float): Alpha parameter (default: 0).
        beta (float): Beta parameter (default: 1).

    Returns:
        numpy.ndarray: The origin-destination matrix.
    """
    print('Computing origin-destination matrix via radiation model\n')

    n = len(spatial_tessellation)
    od_matrix = np.zeros((n, n))

    for id_i in tqdm(spatial_tessellation):
        lat_i, lng_i, m_i = spatial_tessellation[id_i]['lat'], spatial_tessellation[id_i]['lon'], \
                            spatial_tessellation[id_i]['relevance']

        edges = []
        probs = []

        normalization_factor = 1.0 / (1.0 - m_i / M)

        destinations_and_distances = []
        for id_j in spatial_tessellation:
            if id_j != id_i:
                lat_j, lng_j, d_j = spatial_tessellation[id_j]['lat'], spatial_tessellation[id_j]['lon'], \
                                    spatial_tessellation[id_j]['relevance']
                destinations_and_distances.append((id_j, earth_distance((lat_i, lng_i), (lat_j, lng_j))))

        destinations_and_distances.sort(key=operator.itemgetter(1))

        sij = 0.0
        for id_j, _ in destinations_and_distances:
            m_j = spatial_tessellation[id_j]['relevance']
            
            if (m_i + sij) * (m_i + sij + m_j) != 0:
                prob_origin_destination = normalization_factor * \
                                          ((m_i + alpha * sij) * m_j) / \
                                          ((m_i + (alpha + beta) * sij) * (m_i + (alpha + beta) * sij + m_j))
            else:
                prob_origin_destination = 0
                
            sij += m_j
            edges.append([id_i, id_j])
            probs.append(prob_origin_destination)
         
        probs = np.array(probs)

        for i, p_ij in enumerate(probs):
            id_i = edges[i][0]
            id_j = edges[i][1]
            od_matrix[id_i][id_j] = p_ij
        
        # Normalization by row
        sum_odm = np.sum(od_matrix[id_i])
        if sum_odm > 0.0:
            od_matrix[id_i] /= sum_odm
            
    return od_matrix

def sorensen_similarity(v1, v2):
    """
    Calculate the Sørensen similarity coefficient between two vectors.

    Args:
        v1 (numpy.ndarray): First vector.
        v2 (numpy.ndarray): Second vector.

    Returns:
        float: Sørensen similarity coefficient.
    """
    v1 = np.array(v1)
    v2 = np.array(v2)
    numerator = 2 * np.sum(np.minimum(v1, v2))
    denominator = np.sum(v1 + v2)
    return numerator / denominator

def cpc(m1, m2):
    """
    Calculate the Common Part of Commuters (CPC) between two matrices.

    Args:
        m1 (numpy.ndarray): First matrix.
        m2 (numpy.ndarray): Second matrix.

    Returns:
        float: Common Part of Commuters.
    """
    m1 = pd.DataFrame(m1)
    m2 = pd.DataFrame(m2)
    nrow = m1.shape[0]
    ncol = m1.shape[1]
    similarities = []
    for i in range(ncol):
        v1 = m1.iloc[:, i]
        v2 = m2.iloc[:, i]
        similarities.append(sorensen_similarity(v1, v2))
    return np.mean(similarities)
    
def scoring_function(x, y, flow):
    """
    Calculate the scoring function based on the given parameters.

    Args:
        x (float): Alpha parameter.
        y (float): Beta parameter.
        flow (pandas.DataFrame): Flow data.

    Returns:
        float: Scoring value.
    """
    file_path = os.path.join(r"shenzhen_od", f"shenzhen_{round(x, 2)}_{round(y, 2)}.pkl")
    if os.path.exists(file_path):
        with open(file_path, 'rb') as f:
            radiation_matrix = pickle.load(f)
    else:
        radiation_matrix = radiation_od_matrix(spatial_tessellation, alpha=x, beta=y)  
        with open(file_path, 'wb') as f:
            pickle.dump(radiation_matrix, f)

    home_df = flow.groupby("origin", as_index=False)["flow"].sum()
    origin_list = home_df["origin"].values
    num_list = home_df["flow"].values

    home_list = []
    work_list = []
    for i in tqdm(range(len(home_df))):
        home = origin_list[i]
        for j in range(num_list[i]):
            home_list.append(int(home))
            work_list.append(weighted_random_selection(radiation_matrix[int(home)]))

    synthetic_flow = pd.DataFrame()
    synthetic_flow["origin"] = home_list
    synthetic_flow["destination"] = work_list
    synthetic_flow["flows"] = 1

    fake_flow = synthetic_flow.groupby(["origin", "destination"], as_index=False, group_keys=False).count()
    fake_flow["origin"] = fake_flow["origin"].astype("str")
    fake_flow["destination"] = fake_flow["destination"].astype("str")
    fake_flow.rename(columns={"flows":"flow"}, inplace=True)

    flow["origin"] = flow["origin"].astype("str")
    flow["destination"] = flow["destination"].astype("str")

    real_flow = flow.groupby("flow", as_index=False).count()[["flow", "origin"]]

    gravity_flow = fake_flow.groupby("flow", as_index=False).count()[["flow", "origin"]]
    xy = real_flow.merge(gravity_flow, on=["flow"], how="outer")[["origin_x", "origin_y"]].fillna(0.).values

    m = cpc(xy[:, 0], xy[:, 1])
    
    return m

def weighted_random_selection(weights):
    """
    Perform weighted random selection based on the given weights.

    Args:
        weights (numpy.ndarray): Array of weights.

    Returns:
        int: Selected index.
    """
    return np.searchsorted(np.cumsum(weights)[:-1], np.random.random())


In [112]:
# Function to convert a matrix to a flow DataFrame
def matrix2flow(od_matrix):
    origin, destination, flow = [], [], []
    for current_pos, row in od_matrix.items():
        for next_pos, value in row.items():
            origin.append(current_pos)
            destination.append(next_pos)
            flow.append(value)
    
    flow_df = pd.DataFrame({"origin": origin, "destination": destination, "flow": flow})
    return flow_df

# Filtering data
data = data[data["uid"] <= 2000]

# Reshaping mobility data
mobility = data['tile'].values.reshape(-1, 24*7)

# Initializing defaultdicts for the matrices
hw_matrix, other_matrix = defaultdict(lambda: defaultdict(int)), defaultdict(lambda: defaultdict(int))

# Constructing matrices from mobility data
for row in tqdm(range(len(mobility))):
    for i in range(len(mobility[row])-1):
        current_pos, next_pos = mobility[row][i], mobility[row][i+1]
        current_act, next_act = activity_sequence[row][i], activity_sequence[row][i+1]
        if current_pos != next_pos:
            if (current_act == 'H' and next_act == 'W') or (current_act == 'W' and next_act == 'H'):
                hw_matrix[current_pos][next_pos] += 1
            else:
                other_matrix[current_pos][next_pos] += 1

# Converting matrices to flow DataFrames
hw_flow = matrix2flow(hw_matrix)
other_flow = matrix2flow(other_matrix)


100%|████████████████████████████████████████████████████████████████████████████| 2001/2001 [00:00<00:00, 8358.61it/s]


In [113]:
def search_parameter(flow):
    best_alpha, best_beta, best_score = 0., 0, 0.0
    alpha_ls, beta_ls, score_ls = [], [], []

    for alpha in np.arange(0.00, 1.01, 0.1):
        for beta in np.arange(0.00, 1.01, 0.01):
            if alpha + beta <= 1.0:
                score = func(x, y, real_flow)

                alpha_ls.append(x)
                beta_ls.append(y)
                score_ls.append(score)

                if score > best_score:
                    best_alpha, best_beta, best_score = alpha, beta, score
                    print(best_alpha, best_beta, best_score)
    return best_alpha, best_beta, best_score

# hw_alpha, hw_beta, hw_score = search_parameter(hw_flow)
other_alpha, other_beta, other_score = search_parameter(other_flow)

100%|█████████████████████████████████████████████████████████████████████████████| 988/988 [00:00<00:00, 19041.54it/s]


0.0 0.0 0.8058383417147063


100%|█████████████████████████████████████████████████████████████████████████████| 988/988 [00:00<00:00, 19799.96it/s]


0.0 0.01 0.82763671875


100%|█████████████████████████████████████████████████████████████████████████████| 988/988 [00:00<00:00, 19043.12it/s]


...

100%|█████████████████████████████████████████████████████████████████████████████| 988/988 [00:00<00:00, 18119.92it/s]


100%|█████████████████████████████████████████████████████████████████████████████| 988/988 [00:00<00:00, 18690.20it/s]


0.6000000000000001 0.28 0.8464891041162228


100%|█████████████████████████████████████████████████████████████████████████████| 988/988 [00:00<00:00, 17081.01it/s]


100%|█████████████████████████████████████████████████████████████████████████████| 988/988 [00:00<00:00, 19412.25it/s]
